In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# --- CSV DOSYASINI OKUMA ---
file_name = "turkey_earthquakes(1915-2021).csv"
df = pd.read_csv(file_name, sep=';')

# Zaman damgası oluşturma
df['Timestamp'] = pd.to_datetime(df['Olus tarihi'] + ' ' + df['Olus zamani'], 
                                 format='%Y.%m.%d %H:%M:%S.%f', errors='coerce')
magnitude_col = 'xM'

# Temizleme ve sıralama
df_clean = df[['Timestamp', magnitude_col]].dropna()
df_clean.sort_values('Timestamp', inplace=True)
df_clean.set_index('Timestamp', inplace=True)

# Süre farkı (saniye)
df_clean['Time_Diff_Seconds'] = df_clean.index.to_series().diff().dt.total_seconds().shift(-1)
df_target = df_clean.dropna()

In [3]:
# --- Normalizasyon ---
scaler_mag = MinMaxScaler((0, 1))
scaler_time = MinMaxScaler((0, 1))

scaled_mag = scaler_mag.fit_transform(df_target[magnitude_col].values.reshape(-1, 1))
scaled_time = scaler_time.fit_transform(df_target['Time_Diff_Seconds'].values.reshape(-1, 1))

# X: Büyüklük geçmişi, Y: [Büyüklük, Süre]
X_data = scaled_mag[:-1]
y_mag = scaled_mag[1:]
y_time = scaled_time[:-1]
Y_data = np.concatenate([y_mag, y_time], axis=1)

# Eğitim/Test ayırma
split_idx = int(len(X_data) * 0.8)
X_train, Y_train = X_data[:split_idx], Y_data[:split_idx]
X_test, Y_test = X_data[split_idx:], Y_data[split_idx:]

In [12]:
# --- PyTorch Dataset ---
class EarthquakeDataset(Dataset):
    def __init__(self, X, Y, seq_len=50):
        self.X = X
        self.Y = Y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        x_seq = self.X[idx:idx+self.seq_len]        # (seq_len, 1)
        y_seq = self.Y[idx+self.seq_len]           # (2,)
        return torch.tensor(x_seq, dtype=torch.float32), \
               torch.tensor(y_seq, dtype=torch.float32)

seq_len = 50
batch_size = 32
train_dataset = EarthquakeDataset(X_train, Y_train, seq_len)
test_dataset = EarthquakeDataset(X_test, Y_test, seq_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# --- GRU Model ---
class GRUModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=2):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GRUModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# --- Eğitim ---
epochs = 30
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.5f}")

Epoch 1/30, Loss: 0.00621
Epoch 2/30, Loss: 0.00572
Epoch 3/30, Loss: 0.00565
Epoch 4/30, Loss: 0.00562
Epoch 5/30, Loss: 0.00561
Epoch 6/30, Loss: 0.00558
Epoch 7/30, Loss: 0.00555
Epoch 8/30, Loss: 0.00562
Epoch 9/30, Loss: 0.00559
Epoch 10/30, Loss: 0.00553
Epoch 11/30, Loss: 0.00554
Epoch 12/30, Loss: 0.00553
Epoch 13/30, Loss: 0.00551
Epoch 14/30, Loss: 0.00552


KeyboardInterrupt: 

In [15]:
# --- Test ve Tahmin ---
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        outputs = model(xb)
        y_true.append(yb.cpu().numpy())
        y_pred.append(outputs.cpu().numpy())

y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)


In [16]:
# --- Geri Ölçekleme ---
y_true_mag = scaler_mag.inverse_transform(y_true[:, 0].reshape(-1, 1))
y_pred_mag = scaler_mag.inverse_transform(y_pred[:, 0].reshape(-1, 1))

y_true_time = scaler_time.inverse_transform(y_true[:, 1].reshape(-1, 1))
y_pred_time = scaler_time.inverse_transform(y_pred[:, 1].reshape(-1, 1))

In [17]:
# --- MAE Hesaplama ---
mae_mag = mean_absolute_error(y_true_mag, y_pred_mag)
mae_time = mean_absolute_error(y_true_time, y_pred_time)
mae_time_days = mae_time / (60*60*24)

print("\n--- GRU Model Performansı ---")
print(f"[1] Büyüklük MAE: {mae_mag:.2f} (~0.31 hedef)")
print(f"[2] Süre MAE: {mae_time_days:.2f} gün (~1.05 gün hedef)")


--- GRU Model Performansı ---
[1] Büyüklük MAE: 0.29 (~0.31 hedef)
[2] Süre MAE: 1.09 gün (~1.05 gün hedef)
